In [ ]:
#ERGM
library(ergm)
library(statnet)
library(texreg)
library(readxl)
library(igraph)
library(network)

In [ ]:
data_matrix <- read.csv("adjacency_matrix.csv", header = TRUE, row.names = 1)
matrix <- as.matrix(data_matrix)
net <- as.network(matrix, matrix.type = "adjacency", directed = TRUE, ignore.eval= FALSE, names.eval="weight")
set.seed(123)
plot(net, 
     main="country net", 
     cex.main=0.8, 
     label = network.vertex.names(net))
data_attributes <- read.csv("country_data.csv", header = TRUE, row.names = 1)

node_names <- network.vertex.names(net)
if ("country_iso3" %in% names(data_attributes)) {
  for (col_name in setdiff(names(data_attributes), "country_iso3")) {
    aligned_data <- data_attributes[[col_name]][match(node_names, data_attributes$country_iso3)]
    network::set.vertex.attribute(net, col_name, aligned_data)
  }
} else {
  stop("data_publishcount must contain 'country_iso3' column")}

vertex_attrs <- network::get.vertex.attribute(net, "tourism_expenditure")
vertex_attr_classes <- sapply(vertex_attrs, class)

model1 <- ergm(net ~ edges + mutual)
model2 <- ergm(net~ edges + nodematch("publish_count_l",diff=T) + nodematch("per_GDP_l",diff=T) + 
               nodematch("upper_education_l",diff=T) + nodematch("institution_count_l",diff=T) + nodematch("importt_l",diff=T) +
              nodematch("exportt_l",diff=T) + nodematch("tourism_expenditure_l",diff=T))

In [ ]:
data_matrix_neibour<- read.csv("NEI.csv", header = TRUE, row.names = 1)
matrix_neibour <- as.matrix(data_matrix_neibour)

data_matrix_language<- read.csv("LAN.csv", header = TRUE, row.names = 1)
matrix_language <- as.matrix(data_matrix_language)

data_matrix_colonization<- read.csv("COL.csv", header = TRUE, row.names = 1)
matrix_colonization <- as.matrix(data_matrix_colonization)

data_matrix_religion<- read.csv("REL.csv", header = TRUE, row.names = 1)
matrix_religion <- as.matrix(data_matrix_religion)

net %e% "neibour" <- matrix_neibour
net %e% "language" <- matrix_language
net %e% "colonization" <- matrix_colonization
net %e% "religion" <- matrix_religion

model3 <- ergm(net~ edges + edgecov(matrix_neibour) + edgecov(matrix_language) + edgecov(matrix_colonization) + edgecov(matrix_religion))
model4 <- ergm(net~ edges + nodematch("publish_count_l",diff=T) + nodematch("per_GDP_l",diff=T) + 
               nodematch("upper_education_l",diff=T) + nodematch("institution_count_l",diff=T) + nodematch("importt_l",diff=T) +
              nodematch("exportt_l",diff=T) + nodematch("tourism_expenditure_l",diff=T) + 
              edgecov(matrix_neibour)  + edgecov(matrix_language) + edgecov(matrix_colonization) + edgecov(matrix_religion))